In [4]:
import pandas as pd

In [5]:
# Carregamento dos dados
test = pd.read_excel(r'/home/tcduarte/Reps/analise-awareness-python/case_analista_de_business_inteligence.xlsx',
                     sheet_name=1)